In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import urllib.request
from geopy.geocoders import Nominatim
from sklearn.metrics import mean_absolute_error
from datetime import date

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [28]:
def get_dataset(place,output_file_name):

    t = date.today()
    s_date = str(t)
    end_date = s_date[:4] + s_date[5:7] + str(int(s_date[8:])-3)
    start_date = '20000101'
    
    geolocator = Nominatim(user_agent = "Rxghav")
    location = geolocator.geocode(place)
    latitude = location.latitude
    longitude = location.longitude
    url = "https://power.larc.nasa.gov/api/temporal/daily/point?start={}&end={}&latitude={}&longitude={}&community=re&parameters=T2M_MAX%2CT2M_MIN%2CRH2M%2CPRECTOTCORR%2CPS%2CWS10M_RANGE&format=csv&header=true&time-standard=lst".format(start_date,end_date,latitude,longitude)
    output_file_name = output_file_name + '.csv'
    urllib.request.urlretrieve(url,output_file_name)
    return pd.read_csv(output_file_name,skiprows=14)
    
    

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
features_ppt = ['RH2M', 'PS', 'T2M_MAX', 'WS10M_RANGE','T2M_MIN']
target_ptt = 'PRECTOTCORR'
precipitate_model = LinearRegression()

In [43]:
prec_X = test[features_ppt]
prec_Y = test[target_ptt]

In [44]:
x_train_ppt, x_test_ppt, y_train_ppt, y_test_ppt = train_test_split(prec_X, prec_Y, test_size=0.1, random_state=42)


In [45]:
precipitate_model.fit(x_train_ppt,y_train_ppt)

LinearRegression()

In [29]:
loc = input("Enter Name of City for Predicting Weather")
loc_name = loc + 'weather.csv'

Enter Name of City for Predicting WeatherKolkata


In [30]:
test = get_dataset(loc,loc_name)

In [19]:
from datetime import date
t = date.today()

In [20]:
t

datetime.date(2024, 4, 26)

In [23]:
s_date

'2024-04-26'

In [26]:
from datetime import date
t = date.today()

s_date = str(t)
new_date = s_date[:4] + s_date[5:7] + str(int(s_date[8:])-3)


In [27]:
new_date

'20240423'

In [31]:
test.tail()

,YEAR,MO,DY,T2M_MAX,T2M_MIN,RH2M,PRECTOTCORR,PS,WS10M_RANGE
8875,2024,4,19,43.72,26.03,49.69,0.02,100.12,3.90
8876,2024,4,20,45.23,27.15,48.50,0.07,100.08,3.68
8877,2024,4,21,43.58,28.02,50.88,0.21,100.19,4.41
8878,2024,4,22,44.18,29.01,41.25,0.30,100.37,3.52
8879,2024,4,23,43.66,27.30,30.75,0.00,100.31,3.90


In [ ]:
def 

In [21]:
def pre_process_weather_data(weather_df):
    #DATA Preprocessing
    weather_df['YEAR'] = weather_df.YEAR.astype(str)
    weather_df['MO'] = weather_df.MO.astype(str)
    weather_df['DY'] = weather_df.DY.astype(str)

    weather_df['date'] = weather_df.YEAR.str.cat(weather_df['MO'],sep='/')
    weather_df['DATE'] = weather_df.date.str.cat(weather_df['DY'],sep='/')
    
    #removing unrequired attributes
    weather_df.drop(columns=['YEAR','MO','DY','date'],axis=1,inplace=True)
    
    weather_df.set_index(['DATE'],inplace=True)


In [22]:
pre_process_weather_data(test)

In [23]:
test.tail()

,T2M_MAX,T2M_MIN,RH2M,PRECTOTCORR,PS,WS10M_RANGE
DATE,,,,,,
2024/4/19,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2024/4/20,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2024/4/21,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2024/4/22,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2024/4/23,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [9]:
X = test[['RH2M', 'PRECTOTCORR', 'PS', 'WS10M_RANGE']] #input variables (relation)
Y = test[['T2M_MAX', 'T2M_MIN']]  # Two target variables

In [10]:
#train_test split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)


In [11]:
#normalizing data and also converted into np array for ANN INput
scaler_X = MinMaxScaler()
x_train_scaled = scaler_X.fit_transform(x_train)
x_test_scaled = scaler_X.transform(x_test)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [12]:
weather_model = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # Two output neurons for temp_max and temp_min
])

C:\ProgramData\Anaconda3\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
weather_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [14]:
# Train the model
history = weather_model.fit(x_train_scaled, y_train_scaled, epochs=100, batch_size=32, validation_split=0.2, verbose=1,shuffle=True)

Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7958 - loss: 0.0270 - val_accuracy: 0.8957 - val_loss: 0.0219
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8844 - loss: 0.0222 - val_accuracy: 0.8950 - val_loss: 0.0219
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8896 - loss: 0.0215 - val_accuracy: 0.8894 - val_loss: 0.0220
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8885 - loss: 0.0221 - val_accuracy: 0.8975 - val_loss: 0.0219
Epoch 5/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8899 - loss: 0.0220 - val_accuracy: 0.8975 - val_loss: 0.0219
Epoch 6/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8946 - loss: 0.0217 - val_accuracy: 0.8950 - val_loss: 0.0219
Epoch 7/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8915 - loss: 0.0219 - val_accuracy: 0.8919 - val_loss: 0.0219
Epoch 8/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8834 - loss: 0.0213 - val_accu

199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8983 - loss: 0.0214 - val_accuracy: 0.9019 - val_loss: 0.0218
Epoch 52/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8986 - loss: 0.0215 - val_accuracy: 0.8969 - val_loss: 0.0217
Epoch 53/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8986 - loss: 0.0212 - val_accuracy: 0.9045 - val_loss: 0.0217
Epoch 54/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9011 - loss: 0.0217 - val_accuracy: 0.9007 - val_loss: 0.0217
Epoch 55/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8914 - loss: 0.0217 - val_accuracy: 0.9007 - val_loss: 0.0217
Epoch 56/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9001 - loss: 0.0219 - val_accuracy: 0.8988 - val_loss: 0.0217
Epoch 57/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8976 - loss: 0.0214 - val_accuracy: 0.9007 - val_loss: 0.0218
Epoch 58/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8951 - loss: 0.0220 - val_accuracy:

In [15]:
loss = weather_model.evaluate(x_test_scaled, y_test_scaled)
print("Test Loss:", loss)

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8942 - loss: 0.0252
Test Loss: [0.024538660421967506, 0.8936651349067688]


In [16]:
def predict_weather():
    rel_humid = float(input("Enter Relative Humidity: "))
    avg_total_prec = float(input("Enter Average Total Precipitation: "))
    ps_pred = float(input("Enter Pressure Sensor: "))
    winds = float(input("Enter Windspeed: "))
    buff_d = pd.DataFrame({
        'RH2M': [rel_humid],            
        'PRECTOTCORR': [avg_total_prec],
        'PS': [ps_pred],               
        'WS10_RANGE': [winds]                           
    })
    input_preds = scaler_X.transform(buff_d)
    predictions_scaled = weather_model.predict(input_preds)
    

    y_pred = weather_model.predict(x_test_scaled)
    # Assuming y_true contains the true temperature values and y_pred contains the predicted values
    mae = mean_absolute_error(y_test_scaled, y_pred)

    # Calculate error percentage
    error_percent = (mae / y_test_scaled.mean()) 
    
    # Inverse transform the scaled predictions to get the actual temperature values
    predictions = scaler_y.inverse_transform(predictions_scaled)
    predicted_temp_max_next_day = predictions[:, 0] - (predictions[:, 0] * error_percent) 
    predicted_temp_min_next_day = predictions[:, 1] - (predictions[:, 1] * error_percent)

    print("Predicted Max_Temp:", predicted_temp_max_next_day)
    print("Predicted Min_Temp:", predicted_temp_min_next_day)

In [17]:
predict_weather()

Enter Relative Humidity: j


ValueError: could not convert string to float: 'j'

In [ ]:
test.head(2)